# Как подружить PyTotch и C++ и отправить счастливую пару в конечное решение. Используем TorchScript.

Около года назад разработчики PyTorch представили сообществу инструмент, который позволяет с помощью пары строк кода и
нескольких щелчков мыши сделать из пайплайна на питоне отчуждаемое решение, которое можно встроить в систему на  C++ - 
**TorchScript**. Ниже я делюсь опытом его использования и постараюсь описать
встречающиеся на этом пути подводные камни. Особенное внимание уделю реализации проекта на Windows, поскольку, хотя 
исследования в ML обычное делается на Ubuntu, конечное решение часто (внезапно!) требуется под "окошками". 

![](pics/box.png)

TODO Картинка. Кликабельная.

Разработчики PyTorch не обманули. Новый инструмент действительно позволяет превратить исследовательский проект на
PyTorch в код, встраиваемый в систему на С++, за пару рабочих дней, а при некотором навыке и быстрее.

TorchScript появился в PyTorch версии 1.0  и продолжает развиваться и меняться. Если первая версия годичной давности
была полна багов и являлась скорее экспериментальной, то актуальная версия на данный момент версия 1.3 как минимум 
по второму пункту заметно отличается: экспериментальной ее уже не назовешь, она вполне пригодна для практического 
использования. Я буду ориентироваться на нее.

В основе TorchScript лежит собственный автономный (не требующий наличие python) компилятор питон-подобного языка, 
а также средства для конвертации в него программы, написанной на python + PyTorch, методы сохранения и загрузки получившихся
модулей и библиотека для их использования в C++. Для работы придется добавить в проект несколько DLL общим весом около
70MB для работы на CPU и 300MB для GPU версии (это для Windows). TorchScript поддерживает большинство функций PyTorch
и основные возможности языка python. А вот о сторонних библиотеках, таких как OpenCV или NumPy, придется забыть.
К счастью, у многих функций из NumPy есть аналог в pytorch.

## Конвертируем пайплайн на PyTorch модель на TorchScript

TorchScript предлагает два способа преобразования кода на python в его внутренний формат: tracing и scripting
(трассировка и скриптование). Зачем два? Нет, понятно, конечно, что два лучше чем один...

![](pics/4em-boljshe-sdadim-tem-lu4she.jpg)

Но в случае с этими методами получается как в известном афоризме про левый и правый уклон: оба хуже. Что ж, мир
несовершенен. Просто в конкретной ситуации надо выбирать тот, который больше подходит.

Метод трассировки очень прост. Берется некий образец данных (обычно инициализированный случайными числами),
отправляется в интересующую нас функцию или метод класса и PyTorch строит и запоминает граф вычислений примерно так же,
как делает это обычно при обучении нейросети. Вуаля - скрипт готов: 

In [ ]:
import torch
import torchvision
model = torchvision.models.resnet34(pretrained = True)
model.eval()
sample = torch.rand(1, 3, 224, 224)
scripted_model = torch.jit.trace(model, sample)

В примере выше получается объект класса ScriptModule. Его можно сохранить

In [2]:
scripted_model.save('resnet34_script.pth')

и загрузить потом в программу на C++ (об этом ниже) или в код на python вместо исходного объекта:


In [3]:
import cv2
from torchvision.transforms import Compose, ToTensor, Normalize
transforms = Compose([ToTensor(), Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
img = cv2.resize(cv2.imread('pics/cat.jpg'), (224,224))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
x = transforms(img).unsqueeze(0) # add batch dimension

In [4]:
scripted_model = torch.jit.load('resnet34_script.pth')
y = scripted_model(x)

In [5]:
print(y[0].argmax(), y[0][y[0].argmax()])

tensor(282) tensor(12.8130, grad_fn=<SelectBackward>)


Получающийся объект `ScriptModule` может выступать везде, где обычно используется `nn.Module`.

Описанным сповобом можно трассировать экземпляры класса `nn.Module` и функции (в последнем случае получается экземпляр класса `torch._C.Function`)

Этот метод (tracing) имеет важное преимущество: так можно конвертировать почти любой питоновский код, не использующий внешних библиотек. Но есть и не менее важный недостаток: при любых ветвлениях будет запомнена только та ветка, котора исполнялась на тестовых данных:

In [6]:
def my_abs(x):
    if x.max() >= 0:
        return x
    else:
        return -x
my_abs_traced = torch.jit.trace(my_abs, torch.tensor(0))
print(my_abs_traced(torch.tensor(1)), my_abs_traced(torch.tensor(-1)))

c:\miniconda3\lib\site-packages\ipykernel_launcher.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  


tensor(1) tensor(-1)


Упс! Кажется, это не то, что мы хотели бы, правда? Хорошо, что по этому поводу хотя бы выдаётся предупреждающее сообщение
(TracerWarning) Относитесь к таким сообщениям внимательно.

Тут нам на помощь приходит второй метод - scripting:

In [7]:
my_abs_script = torch.jit.script(my_abs)
print(my_abs_script(torch.tensor(1)), my_abs_script(torch.tensor(-1)))

tensor(1) tensor(1)


Ура, ожидаемый результат получен! Scripting рекурсивно анализирует код на python и преобразует в код на собственном языке.
На выходе получаем тот же класс `ScriptModule` (для модулей) или `torch._C.Functio`(для функций) . Казалось бы, счастье есть!
Но возникает другая проблема: внутренний язык TorchScript строго типизированный, в отличие от python.
Тип каждой переменной определяется первым присваиванием, тип аргументов функции по умолчанию - `Tensor`. Поэтому, например, привычный шаблон 

In [8]:
def my_func(x):
    y = None
    if x.max() > 0:
        y = x
    return y

оттассировать не удастся:

In [9]:
my_func = torch.jit.script(my_func)

RuntimeError: 
Variable 'y' previously has type None but is now being assigned to a value of type Tensor
:
at <ipython-input-8-75677614fca6>:4:8
def my_func(x):
    y = None
    if x.max() > 0:
        y = x
        ~ <--- HERE
    return y


Даже точки после констант начинают играть роль:

In [10]:
def my_func(x):
    if x.max() > 0:
        y = 1.25
    else:
        y = 0
    return y
my_func = torch.jit.script(my_func)

RuntimeError: bool value of Tensor with more than one value is ambiguous

Потому что надо писать не `0`, а `0.`, чтобы тип в обеих ветках был одинаковым! Избаловались, понимаешь, со своим питоном!

Это только начало списка тех изменений, которые требуется внести в код на python, чтобы его можно было успешно превратить
в модуль TorchScript. Более подробно самые типичные случаи перечислю чуть позже. В принципе, никакой rocket science тут нет
и свой код вполне можно поправить соответстветствующим образом. А вот исправлять сторонние модули, включая стандартные из
`torchvision`, чаще всего править не хочется, а "как есть" для скриптования они обычно не пригодны.

К счастью, обе технологии можно совмещать: то, что скриптуется - скриптовать, а что не скриптуется - трассировать:

In [84]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.resnet = torchvision.models.resnet34(pretrained = True)
        # без следующих двух строк попытка сделать torch.jit.script(my_module) ниже выдаст ошибку 
        # где-то в недрах resnet34. Поэтому заблаговременно сами заменим self.resnet на ScriptModule.
        self.resnet.eval() # NB: это надо сделать до трассировки! После трассировки не сработает!
        self.resnet = torch.jit.trace(self.resnet, torch.rand((1,3,224,224), dtype=torch.float))
    def forward(self, x):
        if x.shape[2] < 224 or x.shape[3] < 224:
            return torch.tensor(0)
        else:
            return self.resnet(x)
my_module = MyModule()
my_module = torch.jit.script(my_module)

![](pics/Minesweeper - mario.jpg)

![](pics/grabli.jpg)

[![](pics/train_mode.png)](http://qaru.site/questions/16769103/error-when-converting-pytorch-model-to-torchscript)